In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
GEMINI_API_KEY = os.environ["GEMINI_API_KEY"]
GROQ_API_KEY = os.environ["GROQ_API_KEY"]

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS

In [3]:
loader = TextLoader("data/state_of_the_union.txt")
loaded_document = loader.load()

In [5]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
chunks_of_text = text_splitter.split_documents(loaded_document)

In [6]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GEMINI_API_KEY)
vector_db = FAISS.from_documents(chunks_of_text, embeddings)

In [7]:
retriever = vector_db.as_retriever()

In [8]:
response = retriever.invoke("what did he say about Ketanji Brown Jackson?")
response

[Document(id='31032922-6e77-4868-a8fb-3774ef4bafa6', metadata={'source': 'data/state_of_the_union.txt'}, page_content='And my report is this: the State of the Union is strong—because you, the American people, are strong. \n\nWe are stronger today than we were a year ago. \n\nAnd we will be stronger a year from now than we are today. \n\nNow is our moment to meet and overcome the challenges of our time. \n\nAnd we will, as one people. \n\nOne America. \n\nThe United States of America. \n\nMay God bless you all. May God protect our troops.'),
 Document(id='c61fe3f6-de66-478d-bbb7-e052af16cc4f', metadata={'source': 'data/state_of_the_union.txt'}, page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Cons

In [9]:
retriever = vector_db.as_retriever(search_kwargs={"k": 3})
response = retriever.invoke("what did he say about Ketanji Brown Jackson?")
response

[Document(id='31032922-6e77-4868-a8fb-3774ef4bafa6', metadata={'source': 'data/state_of_the_union.txt'}, page_content='And my report is this: the State of the Union is strong—because you, the American people, are strong. \n\nWe are stronger today than we were a year ago. \n\nAnd we will be stronger a year from now than we are today. \n\nNow is our moment to meet and overcome the challenges of our time. \n\nAnd we will, as one people. \n\nOne America. \n\nThe United States of America. \n\nMay God bless you all. May God protect our troops.'),
 Document(id='c61fe3f6-de66-478d-bbb7-e052af16cc4f', metadata={'source': 'data/state_of_the_union.txt'}, page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Cons

## Simple use with LCEL

In [14]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [15]:
template = """ Answer the question based only on the following context: 
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=GEMINI_API_KEY,
)

In [16]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    |prompt
    |llm
    |StrOutputParser()
)

In [18]:
response = chain.invoke("what did he say about Ketanji Brown Jackson?")
response

"He nominated Circuit Court of Appeals Judge Ketanji Brown Jackson 4 days ago. He described her as one of our nation's top legal minds, who will continue Justice Breyer's legacy of excellence."

In [19]:
type(response)  # Should be a string

str